In [ ]:
import tensorflow as tf
from tensorflow import keras

# RESNET34

In [ ]:
class Resnet34():
  def __init__(self):
    pass

  def identity_shortcut(self, inputs, filters):
    x = keras.layers.Conv2D(filters=filters, kernel_size=(3, 3), padding='same', use_bias=False)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)

    x = keras.layers.Conv2D(filters=filters, kernel_size=(3, 3), padding ='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)

    skip_connection = keras.layers.Add()([inputs, x])
    x = keras.layers.Activation('relu')(skip_connection)

    return x

  def projection_shortcut(self, inputs, filters, strides=2):
    x = keras.layers.Conv2D(filters=filters, kernel_size=(3, 3), padding='same', strides=strides, use_bias=False)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)

    x = keras.layers.Conv2D(filters=filters, kernel_size=(3, 3), padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)

    shortcut = keras.layers.Conv2D(filters=filters, kernel_size=(1, 1), padding='same', strides=strides, use_bias=False)(inputs)
    shortcut = keras.layers.BatchNormalization()(shortcut)
    skip_connection = keras.layers.Add()([shortcut, x])
    x = keras.layers.Activation('relu')(skip_connection)

    return x

  def build_model(self, classes, input_shape=(224, 224, 3)):
    inputs = keras.layers.Input(shape=input_shape)

    # Conv1
    x = keras.layers.ZeroPadding2D(padding=3)(inputs)
    x = keras.layers.Conv2D(filters=64, kernel_size=(7, 7), strides=2)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.ZeroPadding2D(padding=1)(x)
    x = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2)(x)

    # Conv2
    x = self.identity_shortcut(x, filters=64)
    x = self.identity_shortcut(x, filters=64)
    x = self.identity_shortcut(x, filters=64)

    # Conv3
    x = self.projection_shortcut(x, filters=128)
    x = self.identity_shortcut(x, filters=128)
    x = self.identity_shortcut(x, filters=128)
    x = self.identity_shortcut(x, filters=128)

    # Conv4
    x = self.projection_shortcut(x, filters=256)
    x = self.identity_shortcut(x, filters=256)
    x = self.identity_shortcut(x, filters=256)
    x = self.identity_shortcut(x, filters=256)
    x = self.identity_shortcut(x, filters=256)
    x = self.identity_shortcut(x, filters=256)

    # Conv5
    x = self.projection_shortcut(x, filters=512)
    x = self.identity_shortcut(x, filters=512)
    x = self.identity_shortcut(x, filters=512)

    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(classes, activation='softmax')(x)

    model = keras.models.Model(inputs=inputs, outputs=outputs)
    return model 

In [ ]:
resnet34 = Resnet34()
resnet34_model = resnet34.build_model(classes=1000)

In [ ]:
resnet34_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_26 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_39 (ZeroPadding  (None, 230, 230, 3)  0          ['input_26[0][0]']               
 2D)                                                                                              
                                                                                                  
 conv2d_147 (Conv2D)            (None, 112, 112, 64  9472        ['zero_padding2d_39[0][0]']      
                                )                                                           

# DENSENET121

In [ ]:
class DenseNet121():
  def __init__(self):
    pass

  def conv_block(self, inputs, k=32):
    x = keras.layers.BatchNormalization()(inputs)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.Conv2D(filters=4*k, kernel_size=(1, 1), use_bias=False)(x)

    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.Conv2D(filters=k, kernel_size=(3, 3), padding='same', use_bias=False)(x)

    skip_connection = keras.layers.Concatenate(axis=-1)([inputs, x])

    return skip_connection

  def dense_block(self, inputs, num_blocks, k=32):
    for i in range(num_blocks):
      x = self.conv_block(inputs, k)
    return x

  def transition_block(self, inputs, fraction=0.5):
    x = keras.layers.BatchNormalization()(inputs)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.Conv2D(filters=inputs.shape[-1]*fraction, kernel_size=(1, 1), use_bias=False)(x)
    x = keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2)(x)
    
    return x

  def build_model(self, classes, input_shape=(224, 224, 3)):
    inputs = keras.layers.Input(shape=input_shape)

    x = keras.layers.Conv2D(kernel_size=(7, 7), filters=64, padding='same', use_bias=False)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=3)(x)

    # Dense block and transition block 1
    x = self.dense_block(x, 6)
    x = self.transition_block(x)

    # Dense block and transition block 2
    x = self.dense_block(x, 12)
    x = self.transition_block(x)

    # Dense block and transition block 3
    x = self.dense_block(x, 24)
    x = self.transition_block(x)

    # Dense block 4
    x = self.dense_block(x, 16)

    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    
    outputs = keras.layers.Dense(classes, activation='softmax')(x)

    model = keras.models.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
densenet121 = DenseNet121()
densenet121_model = densenet121.build_model(1000)

In [ ]:
densenet121_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_31 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_436 (Conv2D)            (None, 224, 224, 64  9408        ['input_31[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_431 (Batch  (None, 224, 224, 64  256        ['conv2d_436[0][0]']             
 Normalization)                 )                                                           